## Import Require Library

In [10]:
import duckdb
import geopandas as gp
import pandas as pd
from shapely.geometry import Point

## Load Data

In [5]:
spatial_data = gp.read_file('../data/Peta Bapernas - Kab/DESA_83218 - Kab.TAB')
poi_data = pd.read_csv('../data/poi/poi_data_from_osm.csv')

C:\Users\diasn\AppData\Local\Temp\ipykernel_11208\2693790467.py:2: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  poi_data = pd.read_csv('../data/poi/poi_data_from_osm.csv')


In [7]:
spatial_data.head(5)

,NO_PROV,NO_KAB,NO_KEC,NO_DES,PROV,KAB_KOT,KEC,KEL_DES,id_kab,geometry
0,11.0,5.0,0.0,0.0,ACEH,ACEH BARAT,,,1105,"POLYGON ((96.49446 4.60135, 96.49445 4.60326, ..."
1,11.0,12.0,0.0,0.0,ACEH,ACEH BARAT DAYA,,,1112,"POLYGON ((96.96627 3.62275, 96.95602 3.62288, ..."
2,11.0,6.0,0.0,0.0,ACEH,ACEH BESAR,,,1106,"MULTIPOLYGON (((95.84115 5.13791, 95.84108 5.1..."
3,11.0,14.0,0.0,0.0,ACEH,ACEH JAYA,,,1114,"MULTIPOLYGON (((96.05169 4.58341, 96.05116 4.5..."
4,11.0,1.0,0.0,0.0,ACEH,ACEH SELATAN,,,1101,"MULTIPOLYGON (((97.90811 2.93116, 97.90811 2.9..."


In [8]:
poi_data.head(5)

,fid,name,name:en,amenity,man_made,shop,tourism,opening_hours,beds,rooms,addr:full,addr:housenumber,addr:street,addr:city,source,name:id,osm_id,osm_type,latitude,longitude
0,1,Klinik Dioratu Medicalindo,NaN,clinic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5036039060,nodes,-1.347399,116.676599
1,2,Puskesmas Pembantu Pejala,NaN,clinic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5036039061,nodes,-1.372694,116.735778
2,3,Puskesmas Petung,NaN,clinic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5036039143,nodes,-1.353585,116.666503
3,4,RM Barokah Denoe,NaN,restaurant,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5035922058,nodes,-1.193041,116.612143
4,5,BRI Unit Sotek,NaN,bank,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5035922059,nodes,-1.192783,116.612422


## Intersection with GeoPandas

In [14]:
# Create Point geometry from longitude and latitude
geometry = [Point(lon, lat) for lon, lat in zip(poi_data['longitude'], poi_data['latitude'])]
poi_data = gp.GeoDataFrame(poi_data, geometry=geometry, crs="EPSG:4326")  # assuming lat/lon is in WGS84 (EPSG:4326)
gdf_joined = gp.sjoin(poi_data, spatial_data, how="inner", predicate="within")

In [15]:
gdf_joined

,fid,name,name:en,amenity,man_made,shop,tourism,opening_hours,beds,rooms,...,index_right,NO_PROV,NO_KAB,NO_KEC,NO_DES,PROV,KAB_KOT,KEC,KEL_DES,id_kab
0,1,Klinik Dioratu Medicalindo,NaN,clinic,NaN,NaN,NaN,NaN,NaN,NaN,...,399,64.0,9.0,0.0,0.0,KALIMANTAN TIMUR,PENAJAM PASER UTARA,,,6409
2,3,Puskesmas Petung,NaN,clinic,NaN,NaN,NaN,NaN,NaN,NaN,...,399,64.0,9.0,0.0,0.0,KALIMANTAN TIMUR,PENAJAM PASER UTARA,,,6409
3,4,RM Barokah Denoe,NaN,restaurant,NaN,NaN,NaN,NaN,NaN,NaN,...,399,64.0,9.0,0.0,0.0,KALIMANTAN TIMUR,PENAJAM PASER UTARA,,,6409
4,5,BRI Unit Sotek,NaN,bank,NaN,NaN,NaN,NaN,NaN,NaN,...,399,64.0,9.0,0.0,0.0,KALIMANTAN TIMUR,PENAJAM PASER UTARA,,,6409
5,6,PKD Gemantar Jumantono,NaN,clinic,NaN,NaN,NaN,NaN,NaN,NaN,...,141,33.0,13.0,0.0,0.0,JAWA TENGAH,KARANGANYAR,,,3313
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154343,154344,Alfarmart Srengsem,NaN,NaN,NaN,travel_agency,NaN,NaN,NaN,NaN,...,179,18.0,71.0,0.0,0.0,LAMPUNG,KOTA BANDAR LAMPUNG,,,1871
154344,154345,Megakamera.com,NaN,NaN,NaN,photo,NaN,NaN,NaN,NaN,...,126,31.0,71.0,0.0,0.0,DKI JAKARTA,JAKARTA PUSAT,,,3171
154345,154346,Sriwijaya Mega Grosir & Mart,NaN,NaN,NaN,supermarket,NaN,NaN,NaN,NaN,...,316,35.0,7.0,0.0,0.0,JAWA TIMUR,MALANG,,,3507
154346,154347,Ocean Dreams Pemuteran,NaN,NaN,NaN,scuba_diving,NaN,NaN,NaN,NaN,...,74,51.0,8.0,0.0,0.0,BALI,BULELENG,,,5108


## Intersection with DuckDB